In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
# SCOPES: Gmail read-only
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
token_path = os.getenv("token_path")
credentials_path = os.getenv("credentials_path")

In [ ]:
def get_message_metadata(service, user_id='me', max_results=5):
    results = service.users().messages().list(userId=user_id, maxResults=max_results, q="after:2025/08/08").execute()
    messages = results.get('messages', [])

    for _, msg in enumerate(messages):
        msg_data = service.users().messages().get(userId=user_id, id=msg['id'], format='metadata', metadataHeaders=['Subject', 'From', 'Date']).execute()
        headers = msg_data.get('payload', {}).get('headers', [])

        email_info = {header['name']: header['value'] for header in headers if header['name'] in ['Subject', 'From', 'Date']}
        print(f"From: {email_info.get('From')}")
        print(f"Subject: {email_info.get('Subject')}")
        print(f"Date: {email_info.get('Date')}")

In [31]:
import pandas as pd
from dags.utils import decode_zip, extract_headers, decode_body 
path_1 = "C:/airflow-docker/data/15-08-2025-10-41.json.gz"
path_2 = "C:/airflow-docker/data/starred-19-08-2025-15-18.json.gz"
decompressed_data_1 = decode_zip(path_1)
decompressed_data_2 = decode_zip(path_2)

df =pd.DataFrame(decompressed_data_1)
df2 =pd.DataFrame(decompressed_data_2)
payload = df.loc[3,["Payload"]].values

In [32]:
#1.5 seconds faster for 250 rows of data
df[["Subject","Sender"]] = df["Payload"].apply(lambda x: pd.Series(extract_headers(x)))
df["Body"] = df["Payload"].apply(decode_body)
df.drop(["Payload"], axis=1, inplace=True)

#1.5 seconds faster for 250 rows of data
df2[["Subject","Sender"]] = df2["Payload"].apply(lambda x: pd.Series(extract_headers(x)))
df2["Body"] = df2["Payload"].apply(decode_body)
df2.drop(["Payload"], axis=1, inplace=True)

In [33]:
df2["l"] = df2["Subject"].apply(lambda x: len(x.split()))

In [34]:
df2.head(12)

,Id,Subject,Sender,Body,l
0,198a74e16ed2573a,your fomo store shipment awb 27722210317645 is...,delhivery limited,order cancelled delivered hey methari satwik y...,8
1,198a8c9b7f0cb680,hpcl 2025 cbt admit card passport size photograph,,dear candidate this has reference to your appl...,8
2,198aeae7ccd54d44,hpcl 2025 cbt admit card passport size photograph,,dear candidate this has reference to your appl...,8
3,198a710359a7ded4,hpcl 2025 cbt admit card,,dear candidate this has reference to your appl...,5
4,1989f70956b3f9f9,hpcl 2025 cbt admit card,,dear candidate this has reference to your appl...,5
5,198a050711a16307,competition launch bigquery ai building the fu...,kaggle,competition launch bigquery ai building the fu...,9
6,198b8ed8322eddfe,hpcl 2025 cbt objection management,,dear candidate this has reference to your appl...,5
7,198ad9d6582b8743,hpcl 2025 cbt admit card,,dear candidate this has reference to your appl...,5
8,198a68bed237dcc2,your fomo store shipment awb 27722210317645 is...,delhivery limited,order placed out for delivery hey methari satw...,10
9,1989a03130a090e5,hpcl 2025 cbt admit card,,dear candidate this has reference to your appl...,5


In [ ]:
df2[df2["l"]>10]
#will cap token size at max_length = 20

,Id,Subject,Sender,Body,l
10,198993660b366a69,your fomo store shipment awb 27722210317645 ha...,delhivery limited,order placed order is on the way hey methari s...,11
18,19882aa34542d70c,sathwik these python api development and web d...,freelancercom,hi sathwikhere are the latest projects matchin...,12
66,19711df7b4240448,your nakpro nutrition shipment awb 19041753916...,delhivery limited,order placed order is on the way hey methari s...,11
67,1970165425959380,your discover express shipment awb 21252041771...,delhivery limited,order placed order is on the way hey methari s...,11
68,196f2601075bd3f6,your bigfoot retail solutions private limited ...,delhivery limited,order cancelled delivered hey methari satwik y...,11
69,196de187aaba325f,your bigfoot retail solutions private limited ...,delhivery limited,order placed order is on the way hey methari s...,14
73,19658abfd4764644,your sr courier shipment awb 19041738132086 ha...,delhivery limited,order placed order is on the way hey methari s...,11
75,195a3f9ad070f24f,your mamaearth surface shipment awb 8609916072...,delhivery limited,order placed order is on the way hey methari s...,11
78,197e5833919083b4,re join ab inbev as a junior data scientist le...,sathwik methari,None,14
118,16efd72f08c0b906,initiation of police verification requests for...,,dear satwik metharithis is to inform you that ...,12


In [38]:
df2.describe()

,l
count,124.000000
mean,5.685484
std,3.183716
min,0.000000
25%,3.000000
50%,5.000000
75%,8.000000
max,15.000000


In [13]:
x = payload[0].get("parts", [])
for h in x:
    print(h)

{'partId': '0', 'mimeType': 'multipart/alternative', 'filename': '', 'headers': [{'name': 'Content-Type', 'value': 'multipart/alternative; boundary="----=_Part_7157979_45136918.1754572226626"'}], 'body': {'size': 0}, 'parts': [{'partId': '0.0', 'mimeType': 'text/plain', 'filename': '', 'headers': [{'name': 'Content-Type', 'value': 'text/plain; charset="utf-8"'}, {'name': 'Content-Transfer-Encoding', 'value': '7bit'}, {'name': 'Content-Disposition', 'value': 'inline'}], 'body': {'size': 67, 'data': 'VGhlIHN0YXR1cyBvZiB5b3VyIGpvYiBhcHBsaWNhdGlvbiBvbiBOYXVrcmkuY29tIGhhcyBiZWVuIHVwZGF0ZWQNCg=='}}, {'partId': '0.1', 'mimeType': 'text/html', 'filename': '', 'headers': [{'name': 'Content-Type', 'value': 'text/html; charset=UTF-8'}, {'name': 'Content-Transfer-Encoding', 'value': 'quoted-printable'}, {'name': 'Content-Disposition', 'value': 'inline'}], 'body': {'size': 78466, 'data': 'PCFET0NUWVBFIGh0bWwgUFVCTElDICItLy9XM0MvL0RURCBYSFRNTCAxLjAgVHJhbnNpdGlvbmFsLy9FTiIgImh0dHA6Ly93d3cudzMub3JnL1R

In [5]:
from dags.utils import extract_headers, decode_body 
for payload in decompressed_data_1["Payload"][19:20]:
    subject, sender = extract_headers(payload)
    body_text = decode_body(payload)

    print(f"\n--- Email ---")
    print(f"From: {sender}")
    print(f"Subject: {subject}")
    print(f"Body:\n{body_text}")


--- Email ---
From: 
Subject: transaction failure for standing instruction on your icici bank debit card
Body:
icici bank online dear customer your payment of inr 11900 towards google play app purchase services with standing instruction xeasctukqj through your icici bank debit card 0647 could not be processed click here to manage your standing instructions sincerely icici bank limited this is a system generated email please do not reply to this email confidentiality information and disclaimer this email message and its attachments may contain confidential proprietary or legally privileged information and is intended solely for the use of the individual or entity to whom it is addressed if you have erroneously received this message please delete it immediately and notify the sender if you are not the intended recipient of the email message you should not disseminate distribute or copy this email email transmission cannot be guaranteed to be secure or errorfree as information could be i

In [14]:
print(len(body_text))

700


In [9]:
import torch
from multiprocessing import cpu_count
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
print(cpu_count())
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())

16
PyTorch version: 2.8.0+cu129
CUDA available: True


In [11]:
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = 2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
out = tokenizer(body_text,
                truncation=True,       
                max_length=512,        
                padding="max_length", 
                return_tensors="pt",
                return_length = True)

In [13]:
print(out["length"])

tensor([512])
